![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/arc/images/microsoft-small-logo.png)
 
## Deploy Azure Arc Data Controller on an existing cluster
 
This notebook walks through the process of deploying a <a href="https://azure.microsoft.com/services/azure-arc/">Azure Arc controller</a> on an existing cluster.
 
* Follow the instructions in the **Prerequisites** cell to install the tools if not already installed.
* The **Required information** will check and prompt you for password if it is not set in the environment variable. The password can be used to access the data controller.

<span style="color:red"><font size="3">Please press the "Run All" button to run the notebook</font></span>

### **Prerequisites** 
Ensure the following tools are installed and added to PATH before proceeding.
 
|Tools|Description|Installation|
|---|---|---|
|kubectl | Command-line tool for monitoring the underlying Kubernetes cluster | [Installation](https://kubernetes.io/docs/tasks/tools/install-kubectl/#install-kubectl-binary-using-native-package-management) |
|Azure CLI (az) | Command-line tool for installing and managing resources in an Azure Arc cluster |[Installation](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli-windows?tabs=azure-cli) |

### **Setup**

In [ ]:
import sys,os,getpass
def run_command(command):
    print("Executing: " + command)
    !{command}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n\t{command}\n')
    print(f'Successfully executed: {command}')

### **Set variables**
Generated by Azure Data Studio using the values collected in the 'Create Azure Arc data controller' wizard.

### **Check dependencies**

In [ ]:
run_command('az --version')

### **Required information**

In [ ]:
if "AZDATA_NB_VAR_ARC_ADMIN_PASSWORD" in os.environ:
    arc_admin_password = os.environ["AZDATA_NB_VAR_ARC_ADMIN_PASSWORD"]
else:
    if arc_admin_password == "":
        arc_admin_password = getpass.getpass(prompt = 'Azure Arc Data Controller password')
        if arc_admin_password == "":
            sys.exit(f'Password is required.')
        confirm_password = getpass.getpass(prompt = 'Confirm password')
        if arc_admin_password != confirm_password:
            sys.exit(f'Passwords do not match.')

### **Set and show current context**

In [ ]:
os.environ["KUBECONFIG"] = arc_config_file
run_command(f'kubectl config use-context {arc_cluster_context}')
run_command('kubectl config current-context')

### **Create Azure Arc Data Controller**

In [ ]:
print (f'Creating Azure Arc Data Controller: {arc_data_controller_name} using configuration {arc_cluster_context}')
os.environ["AZDATA_USERNAME"] = arc_admin_username
os.environ["AZDATA_PASSWORD"] = arc_admin_password
print (f'user: {arc_admin_username} pass: {arc_admin_password}')

if os.name == 'nt':
    print(f'If you don\'t see output produced by az, you can run the following command in a terminal window to check the deployment status:\n\t {os.environ["AZDATA_NB_VAR_KUBECTL"]} get pods -n {arc_data_controller_namespace}')
run_command(f'az arcdata dc create --connectivity-mode Indirect -n {arc_data_controller_name} -k {arc_data_controller_namespace} --subscription {arc_subscription} -g {arc_resource_group} -l {arc_data_controller_location} -sc {arc_data_controller_storage_class} --profile-name {arc_profile} --use-k8s')
print(f'Azure Arc Data Controller: {arc_data_controller_name} created.') 

### **Setting context to created Azure Arc Data Controller**

In [ ]:
# Setting context to Data Controller.
#
run_command(f'kubectl config set-context --current --namespace {arc_data_controller_namespace}')